In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM 
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam

In [5]:
df = pd.read_parquet("small_tokenized_data.parquet")
IDS = df["input_ids"].tolist()
newIDS=[]
for each in IDS:
    #end_index = next((i for i, x in enumerate(each) if x==0), len(each))
    # Remove the trailing 0's
    #each = each[:end_index]
    # Convert the input to a tensor
    input_tensor = torch.tensor(each).unsqueeze(0)
    newIDS.append(each)

In [6]:
bert = BertModel.from_pretrained('bert-base-uncased')

In [10]:
#PyTorch tensor with first 3 reviews from our training set and pass it to it
x = torch.tensor(newIDS[:8])
print(x.shape)
#this is the BERTs final layer output for each token
y, pooled = bert(x, output_all_encoded_layers=False)


# NEXT THING TO DO
#neural net model to go from pooled (vetors) to labels

torch.Size([8, 512])


In [11]:
print(pooled.shape)
print(y.shape)

torch.Size([8, 768])
torch.Size([8, 512, 768])


In [12]:
import torch
# convert our data into a torch dataset
data = torch.utils.data.TensorDataset(pooled)

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

In [34]:
#Where does the 1 and 4 come from 
lin_layer_1 = torch.nn.Linear(8,768)
lin_layer_2 = torch.nn.Linear(768,8)

In [35]:
model = torch.nn.Sequential(   
    lin_layer_1,
    torch.nn.Sigmoid(),
    lin_layer_2,
    torch.nn.Sigmoid()
)

In [36]:
scoring_function = torch.nn.MSELoss()
num_epochs = 1000

optimizer = torch.optim.SGD( model.parameters(), .1 )

In [37]:
for step in range (num_epochs):
    
    for batch in loader:
        #print (batch)
        optimizer.zero_grad()
        batchX = batch[0]
        #batchY = batch[1]
        # do a gradient descent update of out linear model.

        # put data through the model
        preds = model(batchX)

        # collect the error
        #errors = scoring_function(preds, batchY)
        #print(errors)

        # compute the gradient of the error with regard to the model paramenters
        errors.backward()

        # take a step in that direction
        optimizer.step()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x768 and 8x768)

In [38]:
print(batchX.shape)
print(model[0].weight.shape)

torch.Size([8, 768])
torch.Size([768, 8])
